In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [7]:
from sklearn.pipeline import make_pipeline

In [2]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

2023/06/21 02:29:15 INFO mlflow.tracking.fluent: Experiment with name 'green-taxi-duration' does not exist. Creating a new experiment.


<Experiment: artifact_location='gs://taxi-data-mlflow-artifacts/1', creation_time=1687314556104, experiment_id='1', last_update_time=1687314556104, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('../data/green_tripdata_2022-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2022-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [6]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    dv = DictVectorizer()
    model = RandomForestRegressor(**params, n_jobs=-1)

    X_train = dv.fit_transform(dict_train)
    model.fit(X_train, y_train)

    X_val = dv.transform(dict_val)
    y_pred = model.predict(X_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(model, artifact_path="model")

    with open('dict_vectorizer.bin', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('dict_vectorizer.bin')

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.101201727016495


/home/singh/anaconda3/envs/exp_track/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [ ]:
RUN_ID = 'baed3f4f08d64a59b58c51c884a22067'